In [1]:
from __future__ import print_function
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
learning_rate = 1e-3
training_epoch = 50
batch_size = 64
save_epoch = 5
save_path = 'logdir/mnist/'

x=tf.placeholder(dtype=tf.float32,shape=[None,784],name='X_input')
y=tf.placeholder(dtype=tf.float32,shape=[None,10],name='y_input')

W=tf.Variable(tf.random_normal(shape=[784,10]),name='weight')
b=tf.Variable(tf.zeros(shape=[10]),name='bias')

In [3]:
with tf.name_scope('Model'):
    y_hat=tf.nn.softmax(tf.matmul(x,W)+b)
with tf.name_scope('cost'):
    cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_hat,labels=y))
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Acc'):
    acc = tf.equal(tf.argmax(y_hat,1),tf.argmax(y,1))
    acc = tf.reduce_mean(tf.cast(acc,tf.float32))
tf.summary.scalar('loss',cost)
tf.summary.scalar('acc',acc)
mergerd_summary_op = tf.summary.merge_all()

In [4]:
import numpy as np
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    summary_writer = tf.summary.FileWriter(save_path,graph=tf.get_default_graph())
    
    for  epoch in range(training_epoch):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            x_batch,y_batch = mnist.train.next_batch(batch_size)
            _,c,summary = sess.run([optimizer,cost,mergerd_summary_op],
                                   feed_dict={x:x_batch,y:y_batch})
            
            summary_writer.add_summary(summary,epoch * total_batch + i )
            avg_cost += c/total_batch
        if epoch % save_epoch == 0:
            print('Epoch : {:04d},cost is {:.7f}'.format (epoch,avg_cost))
    print('acc',acc.eval({x:mnist.test.images,y:mnist.test.labels}))
                    

Epoch : 0000,cost is 2.2118675
Epoch : 0005,cost is 1.7824185
Epoch : 0010,cost is 1.7513118
Epoch : 0015,cost is 1.7418650
Epoch : 0020,cost is 1.6964381
Epoch : 0025,cost is 1.6308304
Epoch : 0030,cost is 1.6271194
Epoch : 0035,cost is 1.6245624
Epoch : 0040,cost is 1.6226051
Epoch : 0045,cost is 1.6211175
acc 0.8362


In [8]:
!tensorboard --logdir=logdir/mnist/ --host=127.0.0.1

W0808 12:07:22.717541 Reloader plugin_event_accumulator.py:286] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0808 12:07:22.718045 Reloader plugin_event_accumulator.py:294] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
TensorBoard 1.9.0 at http://127.0.0.1:6006 (Press CTRL+C to quit)
^C


- 注意tensorboard使用路径 相对路径根据当前路径决定
- --host --port 可以单独指定访问域名端口等等
- 若出现错误 `no module named tensorflow.tensorboard.tensorboard` conda安装的tensorflow版本 对应其tensorboard所在位置需要确认 特殊情况下可以使用绝对路径下的tensorboard.py 若无 则 main.py 或upgrade pip 后重装tensorflow

### 交叉熵的多种表示方法

- tf.reduce_sum(-y*tf.log(y_hat),axis=1)
- tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(logits=,labels=)
- tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits = ,labels=))